In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# LIST FILES

In [ ]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add
path="./pcaps/"
files_add=find_the_way(path,'.pcap')
files_add

# PCAP2JSON 

In [ ]:
folder("json")
def pcap_parser(oldname):
    piece_size=50000
    new_name=oldname.replace(".pcap","_smaller.pcap")
    new_name=new_name.replace("\\","/")
    command='C:/\"Program Files\"/Wireshark/editcap.exe -c '+str(piece_size)+" \""+oldname+"\" "+new_name   
    os.system(command)
    parsed=find_the_way("./",new_name.split("/")[-1][:-5])
    return parsed,new_name.split("/")[-1][:-5]
def tojson(temp):
    temp=temp.replace("\\","/")
    temp=temp.replace("/","-")
    #print(temp)
    temp=f"./json/{temp[2:-4]}json"
    os.system(f'tshark -r {i} -T json > {temp}')
    #print(temp)

In [ ]:
for i in tqdm(files_add):
    boyut=os.path.getsize(i)
    label_count=0
    if boyut >100000000:
        file_list,keyword=pcap_parser(i)
        for ii in tqdm(file_list):
            tojson(ii)
        #os.remove(i)
    else:tojson(i)

# JSON READER

In [ ]:
import json

# JSON dosyasını okuma fonksiyonu
def read_json_file(file_path):
    with open(file_path, 'rb') as json_file:
        data = json.load(json_file)
    return data

# JSON verisini oku
json_data = read_json_file(temp)
# JSON verisini ekrana yazdır
print(json.dumps(json_data, indent=2))

# JSON2CSV

In [ ]:
path="./json"
files_add=find_the_way(path,'.json')
folder("csv")

In [ ]:
for i in  tqdm(files_add):
    json_data = read_json_file(i)

    # create dataframe
    df = pd.json_normalize(json_data)
    
    temp=i.replace("json","csv")

    df.to_csv(temp, index=False)

In [ ]:
path="./csv"
files_add=find_the_way(path,'.csv')
files_add

In [ ]:
essiz=[]
for i in  tqdm(files_add):
    df=pd.read_csv(i)
    for ii in df.columns:
        if ii not in essiz:
            essiz.append(ii)
            
len(essiz)

In [ ]:
len(essiz)

In [ ]:
path="./csv"
files_add=find_the_way(path,'.csv')

In [ ]:
allsize=[]
for i in tqdm(files_add):
    
    mergedFeatures={"http":[],"dns":[],"mdns":[],"tls":[],"urlencoded":[]}
    prefix={"http":"_source.layers.http.","urlencoded":"_source.layers.urlencoded-form.Form","dns":"_source.layers.dns","mdns":"_source.layers.mdns","tls":"_source.layers.tls"}
    size=[]
    df=pd.read_csv(i)
    df=df.dropna(axis=1, how='all')
    size.append(df.shape[1])
    for ii in prefix:
        for iii in df.columns:
            if prefix[ii] in  iii:
                mergedFeatures[ii].append(iii)
        df[prefix[ii]] = df[mergedFeatures[ii]].apply(
            lambda x: ','.join(x.dropna().astype(str)),
            axis=1
        )
        size.append(df.shape[1])
        for iii in mergedFeatures[ii]:
            del df[iii]

        #if mergedFeatures[ii]!=[]:            print(i,mergedFeatures[ii])

    size.append(df.shape[1])
    df.to_csv(i,index=False)
    allsize.append(size)
for s in allsize:
    print(s)

In [ ]:
essiz=[]
for i in  tqdm(files_add):
    df=pd.read_csv(i)
    for ii in df.columns:
        if ii not in essiz:
            essiz.append(ii)
            
len(essiz)

In [ ]:
for i in essiz:
    print(i)

# CSV MERGER

In [ ]:
flag=0
for i in  tqdm(files_add):
    if flag==0:
        df=pd.read_csv(i)
        df.to_csv("nihai.csv", index=False)
    else:
        df=pd.read_csv("nihai.csv")
        ek=pd.read_csv(i)
        result = pd.concat([df, ek], ignore_index=True, sort=False)
        result=result.dropna(axis=1, how='all')
        result.to_csv("nihai.csv", index=False)
        #result.to_csv(f"nihai{flag}.csv", index=False)
    flag+=1
        
        